In [1]:
# Path setup
import sys
import os

sys.path.append("/home/dchen/Random_Forest_Weights/")

# Basics:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Helpful:
from sklearn.model_selection import train_test_split

# Pipeline and ColumnsTransformer:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# models:
import statsmodels.api as sm

# my functions:
from src_rf.methods.calc_mean import *
from src_rf.methods.calc_weights import *
from src_rf.methods.calc_dist import *

### 0. Setup

In [2]:
def quantile_loss(y_true, y_pred, tau):
    return max(tau * (y_true - y_pred), (1 - tau) * (y_pred - y_true))

quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]

### 1. Load Data

In [3]:
df = pd.read_csv("/home/dchen/Random_Forest_Weights/src_rf/data/energy_data_hourly.csv"
                 , index_col = 'datetime', parse_dates=True)

### 2. Train Test Split

In [4]:
X = df.drop('total_energy_usage', axis = 1)
y = df['total_energy_usage']

# Predetermined random state -> Same as the one used for training the models and all
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3 ,shuffle=False, random_state=42)

# Add intercept, used in quantile regression
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)
X_train_const.drop('Monday', axis = 1, inplace = True)
X_test_const.drop('Monday', axis = 1, inplace = True)

/home/dchen/.conda/envs/Bach_Del_new/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


### 3. Benchmark Models

#### 3.1 Last 100 Benchmark

In [7]:
def baseline_last_100(timestamp, y):
    same_weekday_hour = y[y.index.weekday == timestamp.weekday()]
    same_weekday_hour = same_weekday_hour[same_weekday_hour.index.hour == timestamp.hour]
    latest_100 = same_weekday_hour.iloc[-100:]
    quantiles = latest_100.quantile([0.025, 0.25, 0.5, 0.75, 0.975])
    return quantiles.values

# Making predictions
predictions = [baseline_last_100(timestamp, y_train) for timestamp in y_test.index]

# Store predictions
pred_array = np.array(predictions)

# Compute quantile loss
quantile_levels = [0.025, 0.25, 0.5, 0.75, 0.975]
loss_array = np.zeros((len(y_test), 5))

for idx, (true_val, pred_vals) in enumerate(zip(y_test.values, predictions)):
    for j, (tau, q_val) in enumerate(zip(quantile_levels, pred_vals)):
        loss = quantile_loss(true_val, q_val, tau)
        loss_array[idx, j] = loss

average_loss = loss_array.mean()
print(average_loss)

1.1035525468505574


#### 3.2 Quantile Regression

In [8]:
# Fit models for each quantile using training data with constant term added
qr_models = {}
for q in quantiles:
    qr_model = sm.QuantReg(y_train, X_train_const).fit(q=q)
    qr_models[q] = qr_model

# Predict using each QR model for all test samples with constant term added
qr_predictions = []
for xi in X_test_const.values:
    qr_pred_i = []
    for q in quantiles:
        qr_pred = qr_models[q].predict([xi])
        qr_pred_i.append(qr_pred[0])
    qr_predictions.append(qr_pred_i)

# Convert QR predictions to a numpy array
qr_pred_array = np.array(qr_predictions)

# Compute quantile loss for QR predictions
qr_loss_array = np.zeros((len(y_test), 5))
for idx, (true_val, qr_pred_vals) in enumerate(zip(y_test.values, qr_predictions)):
    for j, (tau, qr_q_val) in enumerate(zip(quantiles, qr_pred_vals)):
        qr_loss = quantile_loss(true_val, qr_q_val, tau)
        qr_loss_array[idx, j] = qr_loss

qr_average_loss = qr_loss_array.mean()
print(qr_average_loss)

1.7924929111488659


### 4. Quantile Random Forest

#### 4.1 Quantile Random Forest 1:

In [9]:
rf_pred_array = np.load('/Data/Delong_BA_Data/rf_weights/energy_quantile_preds.npy')
rf_loss_array = np.zeros((len(y_test), 5))
for count, q in enumerate(quantiles):
    for i in range(y_test.shape[0]):
        rf_loss_array[i,count] = quantile_loss(y_test.values[i], rf_pred_array[i,count], q)
        
rf_average_loss = rf_loss_array.mean()
print(rf_average_loss)

0.7302881297661254


#### 4.2 Quantile Random Forest 2:
Purely time series - all dummy variables: year, month, weekday, hour and (not dummy) count

In [5]:
rf_pred_array_2 = np.load("/Data/Delong_BA_Data/rf_weights/qrf_2/energy_quantile_preds_qrf_2.npy")
rf_loss_array_2 = np.zeros((len(y_test), 5))
for count, q in enumerate(quantiles):
    for i in range(y_test.shape[0]):
        rf_loss_array_2[i,count] = quantile_loss(y_test.values[i], rf_pred_array_2[i,count], q)
        
rf_average_loss = rf_loss_array_2.mean()
print(rf_average_loss)

1.7934436234009783
